# **Northwind Traders - Understanding our Tables**
*Created by : Alfian*

<a id="toc"></a>

<h1 class="list-group-item list-group-item-action active" data-toggle="list" role="tab" aria-controls="home">FOR BETTER EXPERIENCE IN JUPYTER</h1>

In [1]:
# larger cell code
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

***Notes***
* I think my ToC only works in JupyterNotebook, but it's possible that it works in other environment too.
* Hyperlink that I made might not work well with several nbextension like `table of content`
* Others might also causing the issue, but I think uncheck only `table of content` will resolve the issue

<h1 class="list-group-item list-group-item-action active" data-toggle="list" role="tab" aria-controls="home">TABLE OF CONTENT</h1>

<a id="toc"></a>

[1. PURPOSE & INTRODUCTION](#1)<br>
[2. GENERAL DATA INFORMATION](#2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.1. CONTEXT](#201)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2. DATABASE INFORMATION](#202)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3. TABLE RELATIONSHIP](#203)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.4. LIBRARY](#204)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[2.4.1. PACKAGE](#20401)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[2.4.2. DEFINED FUNCTION](#20402)<br>
[3. SQL](#3)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1. CONNECTING TO DATABASE](#301)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.2. CHECK OUT THE DATA WE'VE GOT](#302)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.3. TABLE CREATION 1](#303)<br>

<a id="2"></a>
<font color="lightseagreen" size=+3><b>2. GENERAL DATA INFORMATION</b></font>

<a href="#toc" class="btn btn-primary btn-sm" role="button" aria-pressed="true" 
style="color:white" data-toggle="popover">Table of Contents</a>

<a id="204"></a>
<font color="lightseagreen" size=+2.5><b>2.4. LIBRARY</b></font>

<a href="#toc" class="btn btn-primary btn-sm" role="button" aria-pressed="true" 
style="color:white" data-toggle="popover">Table of Contents</a>

<a id="20401"></a>
<font color="lightseagreen" size=+2.5><b>2.4.1. PACKAGE</b></font>

<a href="#toc" class="btn btn-primary btn-sm" role="button" aria-pressed="true" 
style="color:white" data-toggle="popover">Table of Contents</a>

In [2]:
# to connect SQL with python
import mysql.connector

# general library for data manipulation & data in general
import numpy as np
import pandas as pd

# for visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

# for statistic 
import scipy.stats as stats
from scipy.stats import skew, kurtosis
from scipy.stats import kstest
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal

# show all columns on pandas dataframe
pd.set_option("display.max_columns", None)

# show all value in columns, wider columns
pd.set_option("max_colwidth", 80)
  
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

<a id="20402"></a>
<font color="lightseagreen" size=+2.5><b>2.4.2. DEFINED FUNCTION</b></font>

<a href="#toc" class="btn btn-primary btn-sm" role="button" aria-pressed="true" 
style="color:white" data-toggle="popover">Table of Contents</a>

In [3]:
# get variable name
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [4]:
# needed library for this defined function
from IPython.display import display_html
from IPython.display import IFrame
from itertools import chain,cycle

# displaying column side by side
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw = True)

In [5]:
# silahkan fungsi ini di trial. sepertinya masih butuh beberapa penyesuaian
def general_eda(df):
    gen_info = []
    for i in df.columns:
        try : 
            samples = df[i].drop_duplicates(keep = "first").sample(2).values
        except ValueError : 
            samples = df[i].sample().values

        try : 
            calc_std = round(np.std(df[i]), 4)
            calc_var = round(calc_std ** 2,4)
        except TypeError :
            calc_std = "-"
            calc_var = "-"      

        gen_info.append([
            i,
            df[i].dtype,
            calc_std, 
            calc_var,
            df[i].isna().sum(),
            round(((df[i].isna().sum()*100) / df.shape[0]),2), # cek bagian ini untuk lebih tepatnya.
            df[i].nunique(),
            samples])

    gen_info_df = pd.DataFrame(
                    data = gen_info,
                    columns = ["features", "type", "std", "variance", "n_null", "nullpct", "n_unique", "sampledata"])

    general_eda_check_duplicate = pd.DataFrame(df.duplicated().value_counts()).T

    general_eda_check_duplicate.rename(columns = {False : "not_duplicated", True : "duplicated"}, inplace = True)
    display_side_by_side(gen_info_df,general_eda_check_duplicate, titles = [f"General Information for {namestr(df, globals())[0]}", f"Check for Duplicated data {namestr(df, globals())[0]}"])

<a id="3"></a>
<font color="lightseagreen" size=+3><b>3. SQL</b></font>

<a href="#toc" class="btn btn-primary btn-sm" role="button" aria-pressed="true" 
style="color:white" data-toggle="popover">Table of Contents</a>

<a id="301"></a>
<font color="lightseagreen" size=+2.5><b>3.1. CONNECTING TO DATABASE</b></font>

<a href="#toc" class="btn btn-primary btn-sm" role="button" aria-pressed="true" 
style="color:white" data-toggle="popover">Table of Contents</a>

In [6]:
# create connection
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "password",
    database = "northwind")

# create access to database
mycursor = mydb.cursor()

# Create Function
def sql_df(yourQuery):
    mycursor.execute(yourQuery)
    myResult = mycursor.fetchall()
    df = pd.DataFrame(myResult, columns = mycursor.column_names)
    return df

In [7]:
# trial output
sql_df('''
SHOW TABLES;
''')

,Tables_in_northwind
0,categories
1,customercustomerdemo
2,customerdemographics
3,customers
4,employees
5,employeeterritories
6,orderdetails
7,orders
8,products
9,region


we can see dataframe in output above, this mean we can continue to the next step

<a id="302"></a>
<font color="lightseagreen" size=+2.5><b>3.2. CHECK OUT THE DATA WE'VE GOT</b></font>

<a href="#toc" class="btn btn-primary btn-sm" role="button" aria-pressed="true" 
style="color:white" data-toggle="popover">Table of Contents</a>

Just see all the data we've got from each table

In [8]:
df_categories = sql_df('''
SELECT * FROM categories
''')

df_categories.head(10)

,CategoryID,CategoryName,Description,Picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",b'FFD8FFE000104A46494600010200006400640000FFEC00114475636B790001000400000050...
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, and seasonings",b'FFD8FFE000104A46494600010200006400640000FFEC00114475636B79000100040000004C...
2,3,Confections,"Desserts, candies, and sweet breads",b'FFD8FFE000104A46494600010200006400640000FFEC00114475636B790001000400000041...
3,4,Dairy Products,Cheeses,b'FFD8FFE000104A46494600010200006400640000FFEC00114475636B790001000400000050...
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",b'FFD8FFE000104A46494600010200006400640000FFEC00114475636B79000100040000004C...
5,6,Meat/Poultry,Prepared meats,b'FFD8FFE000104A46494600010200006400640000FFEC00114475636B790001000400000041...
6,7,Produce,Dried fruit and bean curd,b'FFD8FFE000104A46494600010200006400640000FFEC00114475636B790001000400000046...
7,8,Seafood,Seaweed and fish,b'FFD8FFE000104A46494600010200006400640000FFEC00114475636B79000100040000004B...


In [9]:
sql_df('''
SELECT * FROM customercustomerdemo
''')

,CustomerID,CustomerTypeID


In [10]:
sql_df('''
SELECT * FROM customerdemographics
LIMIT 10
''')

,CustomerTypeID,CustomerDesc


In [11]:
df_customers = sql_df('''
SELECT * FROM customers
''')

df_customers.head(10)

,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitucin 2222,Mxico D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquera,Antonio Moreno,Owner,Mataderos 2312,Mxico D.F.,None,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbkp,Christina Berglund,Order Administrator,Berguvsvgen 8,Lule,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
5,BLAUS,Blauer See Delikatessen,Hanna Moos,Sales Representative,Forsterstr. 57,Mannheim,None,68306,Germany,0621-08460,0621-08924
6,BLONP,Blondesddsl pre et fils,Frdrique Citeaux,Marketing Manager,"24, place Klber",Strasbourg,None,67000,France,88.60.15.31,88.60.15.32
7,BOLID,Blido Comidas preparadas,Martn Sommer,Owner,"C/ Araquil, 67",Madrid,None,28023,Spain,(91) 555 22 82,(91) 555 91 99
8,BONAP,Bon app',Laurence Lebihan,Owner,"12, rue des Bouchers",Marseille,None,13008,France,91.24.45.40,91.24.45.41
9,BOTTM,Bottom-Dollar Markets,Elizabeth Lincoln,Accounting Manager,23 Tsawassen Blvd.,Tsawassen,BC,T2F 8M4,Canada,(604) 555-4729,(604) 555-3745


In [12]:
df_employeees = sql_df('''
SELECT * FROM employees
''')

df_employeees

,EmployeeID,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Notes,ReportsTo,PhotoPath,Salary
0,1,Davolio,Nancy,Sales Representative,Ms.,1948-12-08,1992-05-01,507 - 20th Ave. E.Apt. 2A,Seattle,WA,98122,USA,(206) 555-9857,5467,Education includes a BA in psychology from Colorado State University in 1970...,2.0,http://accweb/emmployees/davolio.bmp,2954.55
1,2,Fuller,Andrew,"Vice President, Sales",Dr.,1952-02-19,1992-08-14,908 W. Capital Way,Tacoma,WA,98401,USA,(206) 555-9482,3457,Andrew received his BTS commercial in 1974 and a Ph.D. in international mark...,NaN,http://accweb/emmployees/fuller.bmp,2254.49
2,3,Leverling,Janet,Sales Representative,Ms.,1963-08-30,1992-04-01,722 Moss Bay Blvd.,Kirkland,WA,98033,USA,(206) 555-3412,3355,Janet has a BS degree in chemistry from Boston College (1984). She has also...,2.0,http://accweb/emmployees/leverling.bmp,3119.15
3,4,Peacock,Margaret,Sales Representative,Mrs.,1937-09-19,1993-05-03,4110 Old Redmond Rd.,Redmond,WA,98052,USA,(206) 555-8122,5176,Margaret holds a BA in English literature from Concordia College (1958) and ...,2.0,http://accweb/emmployees/peacock.bmp,1861.08
4,5,Buchanan,Steven,Sales Manager,Mr.,1955-03-04,1993-10-17,14 Garrett Hill,London,None,SW1 8JR,UK,(71) 555-4848,3453,"Steven Buchanan graduated from St. Andrews University, Scotland, with a BSC ...",2.0,http://accweb/emmployees/buchanan.bmp,1744.21
5,6,Suyama,Michael,Sales Representative,Mr.,1963-07-02,1993-10-17,Coventry House\r\nMiner Rd.,London,None,EC2 7JR,UK,(71) 555-7773,428,"Michael is a graduate of Sussex University (MA, economics, 1983) and the Uni...",5.0,http://accweb/emmployees/davolio.bmp,2004.07
6,7,King,Robert,Sales Representative,Mr.,1960-05-29,1994-01-02,Edgeham Hollow\r\nWinchester Way,London,None,RG1 9SP,UK,(71) 555-5598,465,Robert King served in the Peace Corps and traveled extensively before comple...,5.0,http://accweb/emmployees/davolio.bmp,1991.55
7,8,Callahan,Laura,Inside Sales Coordinator,Ms.,1958-01-09,1994-03-05,4726 - 11th Ave. N.E.,Seattle,WA,98105,USA,(206) 555-1189,2344,Laura received a BA in psychology from the University of Washington. She ha...,2.0,http://accweb/emmployees/davolio.bmp,2100.50
8,9,Dodsworth,Anne,Sales Representative,Ms.,1966-01-27,1994-11-15,7 Houndstooth Rd.,London,None,WG2 7LT,UK,(71) 555-4444,452,Anne has a BA degree in English from St. Lawrence College. She is fluent in...,5.0,http://accweb/emmployees/davolio.bmp,2333.33


In [13]:
df_empl_terr = sql_df('''
SELECT * FROM employeeterritories
''')

df_empl_terr

,EmployeeID,TerritoryID
0,2,01581
1,2,01730
2,2,01833
3,2,02116
4,2,02139
5,2,02184
6,5,02903
7,9,03049
8,9,03801
9,1,06897


In [14]:
df_order_details = sql_df('''
SELECT * FROM orderdetails
''')

df_order_details

,OrderID,ProductID,UnitPrice,Quantity,Discount
0,10248,11,14.0000,12,0.0
1,10248,42,9.8000,10,0.0
2,10248,72,34.8000,5,0.0
3,10249,14,18.6000,9,0.0
4,10249,51,42.4000,40,0.0
...,...,...,...,...,...
2150,11077,64,33.2500,2,0.0
2151,11077,66,17.0000,1,0.0
2152,11077,73,15.0000,2,0.0
2153,11077,75,7.7500,4,0.0


In [15]:
df_orders = sql_df('''
SELECT * FROM orders
''')

df_orders

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.3800,Vins et alcools Chevalier,59 rue de l-Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.6100,Toms Spezialitten,Luisenstr. 48,Mnster,None,44087,Germany
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.8300,Hanari Carnes,"Rua do Pao, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.3400,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.3000,Suprmes dlices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,11073,PERIC,2,1998-05-05,1998-06-02,NaT,2,24.9500,Pericles Comidas clsicas,Calle Dr. Jorge Cash 321,Mxico D.F.,None,5033,Mexico
826,11074,SIMOB,7,1998-05-06,1998-06-03,NaT,2,18.4400,Simons bistro,Vinbltet 34,Kobenhavn,None,1734,Denmark
827,11075,RICSU,8,1998-05-06,1998-06-03,NaT,2,6.1900,Richter Supermarkt,Starenweg 5,Genve,None,1204,Switzerland
828,11076,BONAP,4,1998-05-06,1998-06-03,NaT,2,38.2800,Bon app-,"12, rue des Bouchers",Marseille,None,13008,France


In [16]:
df_products = sql_df('''
SELECT * FROM products
''')

df_products.head(10)

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.0000,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.0000,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.0000,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.0000,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.3500,0,0,0,1
5,6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25.0000,120,0,25,0
6,7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.0000,15,0,10,0
7,8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40.0000,6,0,0,0
8,9,Mishi Kobe Niku,4,6,18 - 500 g pkgs.,97.0000,29,0,0,1
9,10,Ikura,4,8,12 - 200 ml jars,31.0000,31,0,0,0


In [17]:
df_region = sql_df('''
SELECT * FROM region
''')

df_region.head(10)

,RegionID,RegionDescription
0,1,Eastern
1,2,Westerns
2,3,Northern
3,4,Southern


In [18]:
df_shippers = sql_df('''
SELECT * FROM shippers
''')

df_shippers

,ShipperID,CompanyName,Phone
0,1,Speedy Express,(503) 555-9831
1,2,United Package,(503) 555-3199
2,3,Federal Shipping,(503) 555-9931


In [19]:
df_suppliers = sql_df('''
SELECT * FROM suppliers
''')

df_suppliers.head(10)

,SupplierID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,None,EC1 4SD,UK,(171) 555-2222,None,None
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,None,#CAJUN.HTM#
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,MI,48104,USA,(313) 555-5735,(313) 555-3349,None
3,4,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai\r\nMusashino-shi,Tokyo,None,100,Japan,(03) 3555-5011,None,None
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Export Administrator,Calle del Rosal 4,Oviedo,Asturias,33007,Spain,(98) 598 76 54,None,None
5,6,Mayumi's,Mayumi Ohno,Marketing Representative,92 Setsuko\r\nChuo-ku,Osaka,None,545,Japan,(06) 431-7877,None,Mayumi's (on the World Wide Web)#http://www.microsoft.com/accessdev/sampleap...
6,7,"Pavlova, Ltd.",Ian Devling,Marketing Manager,74 Rose St.\r\nMoonie Ponds,Melbourne,Victoria,3058,Australia,(03) 444-2343,(03) 444-6588,None
7,8,"Specialty Biscuits, Ltd.",Peter Wilson,Sales Representative,29 King's Way,Manchester,None,M14 GSD,UK,(161) 555-4448,None,None
8,9,PB Knckebrd AB,Lars Peterson,Sales Agent,Kaloadagatan 13,Gteborg,None,S-345 67,Sweden,031-987 65 43,031-987 65 91,None
9,10,Refrescos Americanas LTDA,Carlos Diaz,Marketing Manager,Av. das Americanas 12.890,So Paulo,None,5442,Brazil,(11) 555 4640,None,None


In [20]:
df_territories = sql_df('''
SELECT * FROM territories
''')

df_territories.head(10)

,TerritoryID,TerritoryDescription,RegionID
0,01581,Westboro,1
1,01730,Bedford,1
2,01833,Georgetow,1
3,02116,Boston,1
4,02139,Cambridge,1
5,02184,Braintree,1
6,02903,Providence,1
7,03049,Hollis,3
8,03801,Portsmouth,3
9,06897,Wilton,1


<a id="303"></a>
<font color="lightseagreen" size=+2.5><b>3.3. UNDERSTANDING OUR TABLES</b></font>

<a href="#toc" class="btn btn-primary btn-sm" role="button" aria-pressed="true" 
style="color:white" data-toggle="popover">Table of Contents</a>

In [34]:
general_eda(df_orders)
print("\n",df_orders.shape)

,features,type,std,variance,n_null,nullpct,n_unique,sampledata
0,OrderID,int64,239.6002,57408.2558,0,0.00,830,"[10698, 11003]"
1,CustomerID,object,-,-,0,0.00,89,"[TOMSP, OLDWO]"
2,EmployeeID,int64,2.4981,6.2405,0,0.00,9,"[8, 5]"
3,OrderDate,datetime64[ns],-,-,0,0.00,480,"[1998-01-22T00:00:00.000000000, 1998-03-20T00:00:00.000000000]"
4,RequiredDate,datetime64[ns],-,-,0,0.00,454,"[1996-08-28T00:00:00.000000000, 1997-09-18T00:00:00.000000000]"
5,ShippedDate,datetime64[ns],-,-,21,2.53,387,"[1997-05-22T00:00:00.000000000, 1997-10-21T00:00:00.000000000]"
6,ShipVia,int64,0.7792,0.6072,0,0.00,3,"[1, 3]"
7,Freight,object,-,-,0,0.00,799,"[4.3400, 100.6000]"
8,ShipName,object,-,-,0,0.00,90,"[Around the Horn, Familia Arquibaldo]"
9,ShipAddress,object,-,-,0,0.00,89,"[Jardim das rosas n. 32, 55 Grizzly Peak Rd.]"



 (830, 14)


In [33]:
general_eda(df_order_details)
print("\n",df_order_details.shape)

General Information for df_order_details 
 
 
 
 features 
 type 
 std 
 variance 
 n_null 
 nullpct 
 n_unique 
 sampledata 
 
 
 
 
 0 
 OrderID 
 int64 
 241.322 
 58236.3077 
 0 
 0.0 
 830 
 [10290, 10323] 
 
 
 1 
 ProductID 
 int64 
 22.1539 
 490.7953 
 0 
 0.0 
 77 
 [77, 6] 
 
 
 2 
 UnitPrice 
 object 
 - 
 - 
 0 
 0.0 
 116 
 [49.3000, 27.8000] 
 
 
 3 
 Quantity 
 int64 
 19.0176 
 361.6691 
 0 
 0.0 
 55 
 [42, 90] 
 
 
 4 
 Discount 
 float64 
 0.0 
 0.0 
 0 
 0.0 
 1 
 [0.0] 
 
 
 Check for Duplicated data df_order_details 
 
 
 
 not_duplicated 
 
 
 
 
 0 
 2155


 (2155, 5)


In [32]:
general_eda(df_products)
print("\n",df_products.shape)

,features,type,std,variance,n_null,nullpct,n_unique,sampledata
0,ProductID,int64,22.2261,493.9995,0,0.0,77,"[51, 14]"
1,ProductName,object,-,-,0,0.0,77,"[Chef Anton's Gumbo Mix, Tunnbrd]"
2,SupplierID,int64,8.1667,66.695,0,0.0,29,"[17, 13]"
3,CategoryID,int64,2.3794,5.6615,0,0.0,8,"[3, 5]"
4,QuantityPerUnit,object,-,-,0,0.0,70,"[15 - 300 g rounds, 20 - 1 kg tins]"
5,UnitPrice,object,-,-,0,0.0,62,"[21.0500, 43.9000]"
6,UnitsInStock,int64,35.9117,1289.6502,0,0.0,51,"[85, 57]"
7,UnitsOnOrder,int64,22.9903,528.5539,0,0.0,10,"[30, 50]"
8,ReorderLevel,int64,10.8599,117.9374,0,0.0,7,"[15, 5]"
9,Discontinued,int64,0.3051,0.0931,0,0.0,2,"[1, 0]"



 (77, 10)


In [31]:
general_eda(df_categories)
print("\n",df_categories.shape)

General Information for df_categories 
 
 
 
 features 
 type 
 std 
 variance 
 n_null 
 nullpct 
 n_unique 
 sampledata 
 
 
 
 
 0 
 CategoryID 
 int64 
 2.2913 
 5.2501 
 0 
 0.0 
 8 
 [6, 5] 
 
 
 1 
 CategoryName 
 object 
 - 
 - 
 0 
 0.0 
 8 
 [Beverages, Confections] 
 
 
 2 
 Description 
 object 
 - 
 - 
 0 
 0.0 
 8 
 [Breads, crackers, pasta, and cereal, Sweet and savory sauces, relishes, spreads, and seasonings] 
 
 
 3 
 Picture 
 object 
 - 
 - 
 0 
 0.0 
 8 
 [b'...'] 
 
 
 Check for Duplicated data df_categories 
 
 
 
 not_duplicated 
 
 
 
 
 0 
 8


 (8, 4)
